# 📞 Optimización de Horarios de Llamadas en Cobranzas
## Demo Analítica - Proof of Concept

---

### 📊 Contexto del Problema

En la gestión de cobranzas, una de las principales ineficiencias es realizar llamadas en horarios donde los clientes no atienden. Este notebook demuestra cómo usar datos históricos para identificar las franjas horarias más efectivas y generar un plan de acción para reducir llamadas improductivas.

> **Objetivo:** Aumentar la tasa de contacto efectivo mediante un análisis de patrones de respuesta.

### 🛠️ Stack Técnico y Librerías

Este análisis utiliza `Pandas` para la manipulación de datos y `NumPy` para la generación del dataset sintético.

In [ ]:
import pandas as pd
import numpy as np

print("✅ Librerías cargadas correctamente.")

### 📦 Fase 1: Generación de Datos Sintéticos

Para esta demo, simulamos **250 registros históricos** de llamadas para 10 clientes, cubriendo un horario de 08:00 a 21:00 hs. Los resultados se distribuyen de forma realista para reflejar un escenario típico.

> **Nota sobre Privacidad:** Los datos son 100% sintéticos y generados con `numpy.random`. No se utiliza información real de clientes.

In [ ]:
# Configuración de semilla para reproducibilidad
np.random.seed(42)

clientes = range(1001, 1011)
registros = []

# Generar 25 intentos por cliente
for cliente in clientes:
    for _ in range(25):
        hora = np.random.randint(8, 21)
        minuto = np.random.randint(0, 60)
        resultado = np.random.choice(
            ["atendió", "no atendió", "rechazó"],
            p=[0.35, 0.55, 0.10]
        )
        registros.append({
            "cliente_id": cliente,
            "hora": f"{hora:02d}:{minuto:02d}",
            "resultado": resultado
        })

df = pd.DataFrame(registros)

print(f"✅ Dataset generado con {len(df)} registros.")
df.head()

### ⚙️ Fase 2: Feature Engineering

Transformamos los datos crudos para el análisis:

1.  **Segmentación Horaria:** Clasificamos cada llamada en una de tres franjas:
    - **Mañana** (08:00 - 11:59)
    - **Tarde** (12:00 - 17:59)
    - **Noche** (18:00 - 21:00)
2.  **Variable de Atención:** Creamos una columna binaria (`atendida`) que es `True` si el resultado fue "atendió" y `False` en caso contrario.

In [ ]:
# Función para clasificar horarios en franjas
def franja_horaria(hora_str):
    h = int(hora_str.split(":")[0])
    if 8 <= h < 12:
        return "Mañana"
    elif 12 <= h < 18:
        return "Tarde"
    else:
        return "Noche"

# Aplicar transformaciones
df["franja"] = df["hora"].apply(franja_horaria)
df["atendida"] = df["resultado"] == "atendió"

print("✅ Features de franja horaria y atención creadas.")
df.head()

### 📈 Fase 3: Análisis de Patrones

Calculamos la **tasa de atención** para cada cliente en cada franja horaria. Esta métrica es la clave para identificar los mejores horarios.

> **Métrica Clave:** `tasa_atencion = (llamadas atendidas) / (total de intentos)`

In [ ]:
# Agrupar por cliente y franja para calcular intentos y atenciones
resumen = (
    df.groupby(["cliente_id", "franja"])
      .agg(
          intentos=("atendida", "count"),
          atendidas=("atendida", "sum")
      )
      .reset_index()
)

# Calcular la tasa de atención
resumen["tasa_atencion"] = resumen["atendidas"] / resumen["intentos"]

print("✅ Análisis de tasas de atención completado.")
resumen.head(10)

### 🎯 Fase 4: Generación de Recomendaciones

Con las tasas calculadas, seleccionamos para cada cliente la franja horaria con la **mayor probabilidad de atención**.

In [ ]:
# Encontrar la mejor franja por cliente (la de mayor tasa de atención)
mejor_horario = (
    resumen.sort_values("tasa_atencion", ascending=False)
    .groupby("cliente_id")
    .first()
    .reset_index()
)

print("✅ Mejores horarios por cliente identificados.")
mejor_horario

### 💰 Fase 5: Impacto Empresarial y Plan de Acción

Finalmente, traducimos los hallazgos en un plan de acción concreto y cuantificamos el impacto potencial.

In [ ]:
# Calcular métricas de impacto
tasa_actual = df["atendida"].mean()
tasa_optimizada = mejor_horario["tasa_atencion"].mean()
mejora_relativa = (tasa_optimizada - tasa_actual) / tasa_actual

print("=" * 45)
print("      💰 IMPACTO PROYECTADO 💰")
print("=" * 45)
print(f"Tasa de contacto actual (baseline): {tasa_actual:.1%}")
print(f"Tasa proyectada con optimización:   {tasa_optimizada:.1%}")
print(f"Mejora relativa potencial:          +{mejora_relativa:.1%}")
print("=" * 45)

#### Plan de Acción Operativo

In [ ]:
# Generar el plan de acción priorizado
top_clientes = mejor_horario.sort_values("tasa_atencion", ascending=False)

print("=" * 60)
print("🎯 PLAN DE LLAMADAS OPTIMIZADO - RANKING DE PRIORIDAD")
print("=" * 60)

for _, row in top_clientes.iterrows():
    horario_map = {
        "Mañana": "(08:00-12:00)",
        "Tarde": "(12:00-18:00)",
        "Noche": "(18:00-21:00)"
    }
    
    print(
        f"- Cliente {int(row['cliente_id'])}: "
        f"Llamar en {row['franja'].upper()} {horario_map[row['franja']]}. "
        f"(Prob. éxito: {row['tasa_atencion']:.0%}, "
        f"basado en {int(row['intentos'])} intentos)"
    )

print("=" * 60)

### ✅ Conclusiones

Este análisis demuestra que es posible transformar datos simples de llamadas en una estrategia operativa que mejora la eficiencia.

**Próximos Pasos Sugeridos:**
-   Analizar también por día de la semana.
-   Integrar con una base de datos real (SQL).
-   Crear un dashboard interactivo (Streamlit/Plotly).